# 网易云音乐数据获取

调用[网易云音乐AP](https://binaryify.github.io/NeteaseCloudMusicApi/#/)来获取网易云音乐的一些信息

In [8]:
import os
from lxml import etree
import lxml.html
from selenium import webdriver
from user_agent import generate_user_agent

import requests
from tqdm import tqdm
import time
import json

# 获取歌手的top50歌曲

In [143]:
class Artist(object):
    
    def __init__(self,artist_id):
        self.artist_id = artist_id
    
    # 获取该歌手的top50歌曲
    def get_top50_songs(self):
        artist_id = self.artist_id
        url = f'http://localhost:3000/artist/top/song?id={artist_id}'
        r = requests.get(url)
        songs = r.json()['songs']
        song_ids = [i['id'] for i in songs]
        return song_ids
    
    # 获取某首歌的歌词
    def get_lyric(self,song_id):
        lyric_url = f'http://localhost:3000/lyric?id={song_id}'
        r = requests.get(lyric_url)
        lyric = r.json()['lrc']['lyric']
#         print(lyric)
        return lyric
    
    # 获取歌曲名字和歌手名字
    def get_music_detail(self,song_id):
        song_detail = f'http://localhost:3000/song/detail?ids={song_id}'
        r = requests.get(song_detail)
        music_name = r.json()['songs'][0]['name']
        artist = r.json()['songs'][0]['ar'][0]['name']
        return music_name,artist
    
    # 获取音乐url，并下载为mp3文件
    def get_music_url(self,song_id):
        music_url = f'http://localhost:3000/song/url?id={song_id}'
        music_name,artist = self.get_music_detail(song_id)
        r = requests.get(music_url)
        music = r.json()['data'][0]['url']
        content = requests.get(music).content
        with open(f'{artist}_{music_name}.mp3','wb') as f:
            f.write(content)
        return music
    
    
    # 获取该歌手top50歌曲的所有歌词
    def get_lyrics(self):
        lyrics = {}
        song_ids = self.get_top50_songs()
        for song_id in song_ids:
            lyrics[song_id] = self.get_lyric(song_id)
        return lyrics

## 李荣浩

In [144]:
ronghao = Artist(4292)

In [121]:
# 获取李荣浩的top50歌曲歌词
lyrics_lrh = ronghao.get_lyrics()

In [122]:
lyrics_lrh.keys()

dict_keys([1407551413, 1293886117, 133998, 407450223, 518686034, 31654343, 1415566662, 1318234987, 1318235595, 1418319771, 29710981, 1410490187, 27678655, 27731176, 31134193, 29764562, 428375722, 29764563, 1308818966, 31877243, 506092654, 407450218, 31134170, 570334069, 27731174, 31134197, 27731172, 502242110, 27731177, 511920347, 29764561, 1305547844, 29764564, 862130427, 519250024, 486323960, 407459147, 407459148, 515143802, 1309866644, 29764565, 400074326, 1318234982, 27731179, 479223381, 407459145, 493631465, 29764568, 134000, 1318234981])

In [145]:
# 获取李荣浩某首歌的歌名
ronghao.get_music_detail(1293886117)

('年少有为', '李荣浩')

In [146]:
# 获取年少有为的音乐URL，并下载音乐
ronghao.get_music_url(1293886117)

'http://m8.music.126.net/20200206211754/af9f94e92e9d9f6beaed4cd31abcad24/ymusic/030b/060b/035b/4a0c8e3d1c1afd37d18e4165494e9ef2.mp3'